In [642]:
import os
import git
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import ast
import re

In [486]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import cross_val_score

In [352]:
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Input
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers import Bidirectional
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import Reshape
from keras.layers import Average
from keras_contrib.layers import CRF
from sklearn.model_selection import train_test_split
import keras
from sklearn.model_selection import GridSearchCV

In [622]:
pd.set_option('display.max_colwidth', -1)

In [7]:
# get the git repo
def get_git_root(path):
	git_repo = git.Repo(path, search_parent_directories=True)
	git_root = git_repo.git.rev_parse("--show-toplevel")
	return git_root

In [86]:
from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma

In [84]:
# setup env
# setup env
import nltk
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/samarthgoal/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/samarthgoal/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [81]:
## tokenize the text..
from keras.preprocessing.text import Tokenizer
from keras.layers import Embedding
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()
def get_tokens(sentence):
#     tokens = nltk.word_tokenize(sentence)  # now using tweet tokenizer
    tokens = tknzr.tokenize(sentence)
    tokens = [token for token in tokens if (token not in stopwords and len(token) > 1)]
    tokens = [get_lemma(token) for token in tokens]
    return (tokens)

Using TensorFlow backend.


In [677]:
def clean_text(text):
    text = re.sub(r'(https?://\S+)', "", text)
    text = re.sub(r'(\@\w+)', "author",text)
    text.rstrip
    text.lstrip
    return(text)

In [733]:
embeddings_dir  = os.path.join(get_git_root(os.getcwd()),"input","embeddings")
extract_dir = os.path.join(get_git_root(os.getcwd()),"input","hexagon_extract")
annotatted_dir = "/home/samarthgoal/misc_data/annotated_data/"
classifier_dir = os.path.join(get_git_root(os.getcwd()),"models","classifier")
input_dir = os.path.join(get_git_root(os.getcwd()),"input")
model_dir = os.path.join(get_git_root(os.getcwd()),"models")

In [513]:
glove_file = os.listdir(embeddings_dir)[0]
annotatted_files = os.listdir(annotatted_dir)
hexagon_files = [file for file in os.listdir(extract_dir) if file.endswith(".csv")]

### preparing the  data

In [133]:
## extracting the annotated data
df_annotation = pd.DataFrame()
for file in annotatted_files:
    df_ = pd.read_excel(os.path.join(annotatted_dir,file))
    columns = (df_.columns)
    df_ = df_.rename(columns={columns[1]: "label", columns[0]: "tweetText"})
    print(df_.columns)
    frames = [df_,df_annotation]
    df_annotation = pd.concat(frames,ignore_index=True)

Index(['tweetText', 'label'], dtype='object')
Index(['tweetText', 'label'], dtype='object')


In [134]:
columns = (df_annotation.columns)

In [138]:
len(df_annotation)

519

In [146]:
df_annotation = df_annotation.sample(frac=1).reset_index(drop=True)

In [148]:
len(df_annotation)

519

In [187]:
from keras.preprocessing.text import Tokenizer
from keras.layers import Embedding
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()
def get_tokens(sentence):
#     tokens = nltk.word_tokenize(sentence)  # now using tweet tokenizer
    tokens = tknzr.tokenize(sentence)
    tokens = [token for token in tokens if (token not in stopwords and len(token) > 1 and token.isalpha())]
    tokens = [get_lemma(token) for token in tokens]
    return (tokens)

In [206]:
# create the word2vec dict from the dictionary
def get_word2vec(file_path):
    file = open(file_path, "r")
    if (file):
        word2vec = dict()
        split = file.read().splitlines()
        for line in split:
            key = line.split(' ',1)[0] # the first word is the key
            value = np.array([float(val) for val in line.split(' ')[1:]])
            word2vec[key] = value
        return (word2vec)
    else:
        print("invalid fiel path")

In [325]:
class MeanEmbeddingVectorizer(object):
	def __init__(self, word2vec):
		self.word2vec = word2vec
		# if a text is empty we should return a vector of zeros
		# with the same dimensionality as all the other vectors
		self.dim = len(next(iter(word2vec.values())))

	def fit(self, X, y):
		return self

	def transform(self, X):
		return np.array([
			np.mean([self.word2vec[w] for w in words if w in self.word2vec]
					or [np.zeros(self.dim)], axis=0)
			for words in tqdm(X)
		])

In [255]:
le = LabelEncoder()

In [492]:
## tokenize the sentences..
X = list(df_annotation['tweetText'].apply(get_tokens))
y = (list(df_annotation['label']))
y = le.fit_transform(y)

In [290]:
max_len = len(max(token_list, key=len))

In [291]:
# prepare the input: pad and fit
# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(token_list)
vocab_size = len(t.word_index) + 1
vocab_size

2843

In [ ]:
word2vec = get_word2vec(os.path.join(embeddings_dir,embedding_file))

In [455]:
vocab_size = vocab_size
encoded_docs = t.texts_to_sequences(token_list)
# pad documents to a max length of 4 words
max_len = max_len
max_len = 60
X = pad_sequences(encoded_docs, maxlen=max_len, padding='post')
y = y

In [456]:
## train test split
X_train,X_test,Y_train,Y_test  = train_test_split(X, y,test_size =0.20,random_state= 4 )

In [457]:
## get the embedding matrix weights:
from numpy import zeros
embedding_matrix = zeros((vocab_size, 100))
for word, i in t.word_index.items():
    embedding_vector = word2vec.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [458]:
## pass to the bi-lstm model
max_len= 60
input = Input(shape=(max_len,))
model = Embedding(vocab_size,100,weights=[embedding_matrix],input_length=max_len)(input)
model =  Bidirectional (LSTM (100,return_sequences=True,dropout=0.50),merge_mode='concat')(model)
model = TimeDistributed(Dense(100,activation='relu'))(model)
model = Flatten()(model)
model = Dense(100,activation='relu')(model)
output = Dense(3,activation='softmax')(model)

In [459]:
model = Model(input,output)

In [460]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

In [461]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_32 (InputLayer)        (None, 60)                0         
_________________________________________________________________
embedding_34 (Embedding)     (None, 60, 100)           284300    
_________________________________________________________________
bidirectional_29 (Bidirectio (None, 60, 200)           160800    
_________________________________________________________________
time_distributed_28 (TimeDis (None, 60, 100)           20100     
_________________________________________________________________
flatten_24 (Flatten)         (None, 6000)              0         
_________________________________________________________________
dense_76 (Dense)             (None, 100)               600100    
_________________________________________________________________
dense_77 (Dense)             (None, 3)                 303       
Total para

In [462]:
## pass the training data
model.fit(X_train,Y_train,validation_split=0.25, nb_epoch = 10, verbose = 2)

/home/samarthgoal/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  from ipykernel import kernelapp as app


Train on 311 samples, validate on 104 samples
Epoch 1/10
 - 14s - loss: 1.0601 - acc: 0.4405 - val_loss: 1.0292 - val_acc: 0.5577
Epoch 2/10
 - 3s - loss: 0.9096 - acc: 0.6045 - val_loss: 0.9163 - val_acc: 0.5865
Epoch 3/10
 - 4s - loss: 0.7607 - acc: 0.6656 - val_loss: 0.9105 - val_acc: 0.6250
Epoch 4/10
 - 3s - loss: 0.6692 - acc: 0.7235 - val_loss: 0.9828 - val_acc: 0.6250
Epoch 5/10
 - 3s - loss: 0.5839 - acc: 0.7685 - val_loss: 0.8812 - val_acc: 0.6538
Epoch 6/10
 - 3s - loss: 0.4938 - acc: 0.8264 - val_loss: 0.9705 - val_acc: 0.5962
Epoch 7/10
 - 3s - loss: 0.4833 - acc: 0.8071 - val_loss: 1.0160 - val_acc: 0.6635
Epoch 8/10
 - 3s - loss: 0.3975 - acc: 0.8392 - val_loss: 1.0637 - val_acc: 0.5962
Epoch 9/10
 - 3s - loss: 0.3161 - acc: 0.8907 - val_loss: 1.0680 - val_acc: 0.6731
Epoch 10/10
 - 3s - loss: 0.2768 - acc: 0.8939 - val_loss: 1.1217 - val_acc: 0.6346


In [470]:
# evaluate the model
loss, accuracy = model.evaluate(X_test, Y_test, verbose=2)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 75.000000


In [464]:
from sklearn.metrics import classification_report,confusion_matrix
Y_pred = model.predict(X_test)
y_pred = np.array([np.argmax(pred) for pred in Y_pred])
print('  Classification Report:\n',classification_report(Y_test,y_pred),'\n')

  Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.87      0.85        39
           1       0.70      0.57      0.63        28
           2       0.70      0.76      0.73        37

   micro avg       0.75      0.75      0.75       104
   macro avg       0.74      0.73      0.73       104
weighted avg       0.75      0.75      0.75       104
 



In [323]:
# evaluate the model
##with max_len =55
loss, accuracy = model.evaluate(X_test, Y_test, verbose=2)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 77.884615


In [324]:
##with max_len =55
from sklearn.metrics import classification_report,confusion_matrix
Y_pred = model.predict(X_test)
y_pred = np.array([np.argmax(pred) for pred in Y_pred])
print('  Classification Report:\n',classification_report(Y_test,y_pred),'\n')

  Classification Report:
               precision    recall  f1-score   support

           0       0.84      0.95      0.89        39
           1       0.66      0.68      0.67        28
           2       0.81      0.68      0.74        37

   micro avg       0.78      0.78      0.78       104
   macro avg       0.77      0.77      0.76       104
weighted avg       0.78      0.78      0.78       104
 



In [476]:
## dump the classifier(neural model)
import pickle
filename = os.path.join(classifier_dir,"lstm_model.pkl")
with open(filename,"wb") as f:
    pickle.dump(model,f)

### getting the baseline with simple models

In [493]:
vect = MeanEmbeddingVectorizer(word2vec)
X = vect.transform(token_list)
y = y

100%|██████████| 519/519 [00:00<00:00, 21238.54it/s]


In [494]:
X_train,X_test, Y_train, Y_test = train_test_split(X,y,test_size=0.25,random_state =4)

In [495]:
rf = RandomForestClassifier(n_estimators=100, max_depth=2,
                             random_state=0)
rf.fit(X_train,Y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=2, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [496]:
etree = ExtraTreesClassifier(n_estimators=200)
etree.fit(X_train,Y_train)

ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [497]:
def svm_wrapper(X_train,Y_train):
    param_grid = [
    {'C': [1, 10], 'kernel': ['linear']},
    {'C': [1, 10], 'gamma': [0.1,0.01], 'kernel': ['rbf']},]
    svm1 = GridSearchCV(SVC(),param_grid)
    svm1.fit(X_train, Y_train)
    return(svm1)

In [498]:
sv = svm_wrapper(X_train,Y_train)

/home/samarthgoal/anaconda3/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


In [499]:
Y_pred = rf.predict(X_test)
from sklearn.metrics import classification_report,confusion_matrix
print('  Classification Report:\n',classification_report(Y_test,Y_pred),'\n')
accuracy_score(Y_test, Y_pred)

  Classification Report:
               precision    recall  f1-score   support

           0       0.59      0.78      0.67        49
           1       0.27      0.08      0.13        36
           2       0.56      0.69      0.62        45

   micro avg       0.55      0.55      0.55       130
   macro avg       0.48      0.52      0.47       130
weighted avg       0.49      0.55      0.50       130
 



0.5538461538461539

In [500]:
Y_pred = sv.predict(X_test)
print('  Classification Report:\n',classification_report(Y_test,Y_pred),'\n')
accuracy_score(Y_test, Y_pred)

  Classification Report:
               precision    recall  f1-score   support

           0       0.70      0.76      0.73        49
           1       0.56      0.39      0.46        36
           2       0.60      0.69      0.64        45

   micro avg       0.63      0.63      0.63       130
   macro avg       0.62      0.61      0.61       130
weighted avg       0.62      0.63      0.62       130
 



0.6307692307692307

In [502]:
Y_pred = etree.predict(X_test)
print('  Classification Report:\n',classification_report(Y_test,Y_pred),'\n')
accuracy_score(Y_test, Y_pred)

  Classification Report:
               precision    recall  f1-score   support

           0       0.71      0.73      0.72        49
           1       0.46      0.31      0.37        36
           2       0.60      0.73      0.66        45

   micro avg       0.62      0.62      0.62       130
   macro avg       0.59      0.59      0.58       130
weighted avg       0.60      0.62      0.60       130
 



0.6153846153846154

In [503]:
score = cross_val_score(etree,X,y)
score

/home/samarthgoal/anaconda3/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


array([0.53448276, 0.53179191, 0.61046512])

In [504]:
## lets predict with the model
## create a big file of constituents file...

In [520]:
df_tweets = pd.read_csv(os.path.join(extract_dir,hexagon_files[0]),lineterminator="\n")

In [567]:
# retweets
len([ele for ele in list(df_tweets["retweetText"]) if not pd.isnull(ele)])

4917

In [568]:
len(df_tweets)

8698

In [747]:
## merge all of the files
df_final = pd.DataFrame()
for file in hexagon_files:
    filename = os.path.join(extarct_dir,file)
    df = pd.read_csv(filename,lineterminator="\n")
    frames = [df,df_final]
    df_final = pd.concat(frames)

/home/samarthgoal/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:7: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [748]:
df_final = df_final.loc[:, ~df_final.columns.str.contains('^Unnamed')]

In [749]:
## total length of df_final
len(df_final.userID.unique())

4857

In [613]:
## retweets
retweets_count = len([ele for ele in list(df_final["retweetText"]) if not pd.isnull(ele)])
## no of retweets : 63984
print("% of retweets:", (retweets_count / len(df_final))*100) 

% of retweets: 57.50130308967054


In [670]:
## removing the retweets -- we gonna classify the original tweets based on bilstm model
df_temp = df_final.loc[df_final['retweetText'].isnull() == True]  # removing the rewteets
df_original = df_temp.loc[~df_temp['tweetText'].str.startswith('RT', na=False)]   # removing the text that contains retweet

In [682]:
sentence_test = list(df_original['tweetText'].apply(clean_text))

In [683]:
len(list(df_original['tweetText']))

45596

In [751]:
token_list = list(df_final['tweetText'].apply(get_tokens))

In [752]:
len(token_list)

111274

In [704]:
max_len = 60
encoded_docs = t.texts_to_sequences(token_list)
X_new = pad_sequences(encoded_docs, maxlen=max_len, padding='post')

In [705]:
len(X_new)

45596

In [706]:
Y_new = [np.argmax(ele) for ele in model.predict(X_new)]

In [707]:
df_original['label'] = Y_new

/home/samarthgoal/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [714]:
len(df_original.loc[df_original.label == 2])

27255

In [722]:
## news campaign
len(df_original.loc[df_original.label == 0])

12323

In [736]:
df_input_pred  = df_original

In [745]:
len(df_input_pred.userID.unique())

2878

In [743]:
## predicting the mono and poly users
weed_words = pickle.load(open(os.path.join(model_dir, "weed_words.pkl"), "rb"))
weed_words = [(" " + word + " ") for word in weed_words]
pattern_weed = "|".join(weed_words)
pattern_juul = 'juul'
print("extracting the pattern for juul")
df_tweet_weeds = df_input_pred[df_input_pred['tweetText'].str.contains(pattern_weed, case=False)]
index = df_tweet_weeds.index  # the file after filtering have the index contained within
df_weeds = df_input_pred.loc[index]
poly_users = list(set(list(df_weeds.loc[df_weeds.label == 3]['userID'])))
total_users = list(df_input_pred.userID.unique())
poly_length = len(poly_users)
total_users_length = len(total_users)
mono_length = total_users_length - poly_length
print("total users = ", total_users_length)
print("no of poly users = ", poly_length)
print("no of mono users = ", mono_length)
# print("% of poly users is ", poly_length / total_users_length)
# print("% of mono users is ", mono_length / total_users_length)
print("*** starting with the poly sub type users")
poly_user1 = list()
poly_user2 = list()
poly_user3 = list()
poly_und = list()
total_users = list(df_input_pred.userID.unique())
for user in tqdm(poly_users):
    user_tweets = df_input_pred.loc[df_input_pred.userID == user]
    user_tweets.sort_values(by='tweetCreatedAt', ascending=True,
                            inplace=True)  # sort by tweet created at
    juul_tweets = user_tweets[user_tweets['tweetText'].str.contains(pattern_juul, case=False)]
    juul_tweets.reset_index(drop=True, inplace=True)
    if (len(juul_tweets) > 0):
        time_j = pd.to_datetime(
            juul_tweets.head(1)['tweetCreatedAt'].values[0])  # getting the tweet with
    else:
        time_j = None
    weed_tweets = user_tweets[user_tweets['tweetText'].str.contains(pattern_weed, case=False)]
    if (len(weed_tweets) > 0):
        weed_tweets_user = weed_tweets[weed_tweets.label == 3]
        if (len(weed_tweets_user) > 0):
            times_w = pd.to_datetime(list(weed_tweets_user[
                                              'tweetCreatedAt']))
    else:
        times_w = None
    if (time_j != None and times_w is not None):
        pos = list(times_w).index(util.nearest((times_w), time_j))
        if (pos >= len(times_w)):
            poly_user1.append(user)
        elif (pos == 0):
            poly_user2.append(user)
        else:
            poly_user3.append(user)
    else:
        poly_und.append(user)
print("Poly type users calculated")
print("total users =", len(total_users))
print("****************\n")
print("% of pol1 users = ", len(poly_user1) / len(poly_users))
print("\n")
print("% of pol2 users = ", len(poly_user2) / len(poly_users))
print("\n")
print("% of pol3 users = ", len(poly_user3) / len(poly_users))
print("\n")
print("% of undefined users = ", len(poly_und) / len(poly_users))

extracting the pattern for juul


0it [00:00, ?it/s]

total users =  2878
no of poly users =  0
no of mono users =  2878
*** starting with the poly sub type users
Poly type users calculated
total users = 2878
****************



ZeroDivisionError: division by zero

### Dump

In [730]:
## predicted data
filepath = os.path.join(input_dir,"labelled_data")
df_original.to_csv(os.path.join(filepath,"tweets_predicted.csv"))

In [731]:
## annnotated data
filepath = os.path.join(input_dir,"annotated_data")
df_annotation.to_csv(os.path.join(filepath,"annotated_hex.csv"))